# <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">طبقه‌بندی با استخراج ویژگی با HOG</div>

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">لود کتابخانه‌های مورد نیاز</div>

In [1]:
import cv2
import numpy as np

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">لود مجموعه داده</div>

In [2]:
def split2d(img, cell_size, flatten=True):
    h, w = img.shape[:2]
    sx, sy = cell_size
    cells = [np.hsplit(row, w//sx) for row in np.vsplit(img, h//sy)]
    cells = np.array(cells)
    if flatten:
        cells = cells.reshape(-1, sy, sx)
    return cells

def load_digits(file_name):
    digits_img = cv2.imread(file_name, 0)
    digits = split2d(digits_img, (20, 20))
    labels = np.repeat(np.arange(10), len(digits)/10)
    return digits, labels

digits, labels = load_digits('dataset/digits.png')
print(digits.shape)
print(labels.shape)

(5000, 20, 20)
(5000,)


### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">بر زدن مجموعه داده (shuffle کردن)</div>

In [3]:
rand = np.random.RandomState(10)
shuffle = rand.permutation(len(digits))
digits, labels = digits[shuffle], labels[shuffle]

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">ایجاد شئ HOG برای استخراج ویژگی</div>

In [4]:
winSize = (20,20)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradient = True

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradient)

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">استخراج ویژگی‌های تصویر با HOG</div>

In [5]:
hog_descriptors = []
for img in digits:
    hog_descriptors.append(hog.compute(img))
hog_descriptors = np.squeeze(hog_descriptors)
hog_descriptors.shape

(5000, 144)

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">جدا کردن مجموعه داده آموزش و آزمون (train/test)</div>

In [6]:
train_n=int(0.9*len(hog_descriptors))
digits_train, digits_test = np.split(digits, [train_n])
hog_descriptors_train, hog_descriptors_test = np.split(hog_descriptors, [train_n])
labels_train, labels_test = np.split(labels, [train_n])

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">ایجاد یک طبقه‌بند - در اینجا SVM</div>

In [7]:
C=12.5
gamma=0.50625
model = cv2.ml.SVM_create()
model.setGamma(gamma)
model.setC(C)
model.setKernel(cv2.ml.SVM_RBF)
model.setType(cv2.ml.SVM_C_SVC)

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">آموزش طبقه‌بند</div>

In [8]:
model.train(hog_descriptors_train, cv2.ml.ROW_SAMPLE, labels_train)

True

### <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">مشاهده‌ی نتیجه</div>

In [9]:
predictions = model.predict(hog_descriptors_test)[1].ravel()
accuracy = (labels_test == predictions).mean()
print('Percentage Accuracy: %.2f %%' % (accuracy*100))

Percentage Accuracy: 98.60 %


Source and More complete example:
https://github.com/spmallick/learnopencv/blob/master/digits-classification/train_digits.py